```
L'idée :
Diagramme ombrothermique comparant la dernière année enregistrée avec la moyenne des x dernières années.
Pour ça y'a besoin des données journalières de la dernière année et des données mensuelles des dernières années.
```

```
Il vaut mieux que tu crée un tableau avec les valeurs de 2023 et les valeurs moyennes entre 2002 et 2023 plutot que de mixer les deux graphes a la fin
```

In [1]:
import pandas as pd
import os
import pandas as pd
import holoviews as hv
import hvplot.pandas
hv.extension('bokeh')  # Activer l'extension pour afficher les graphiques
output_dir = 'output/'
os.makedirs(output_dir, exist_ok=True)

# Plot données 2023

In [ ]:
data = pd.read_csv(r"data\MENSQ_31_latest-2023-2024.csv", sep=';', decimal=',', thousands='.', index_col=0)  # Lire le fichier CSV
data.head()

In [4]:
data_2023 = data[data['AAAAMM'].astype(str).str.startswith('2023')]   # Filtrer les données pour l'année 2023
data_2023 = data_2023[['AAAAMM', 'RR', 'TM']]    # Sélectionner les colonnes d'intérêt
data_2023['Mois'] = data_2023['AAAAMM'].astype(str).str[4:6].astype(int)   # Extraire le mois
data_2023 = data_2023.sort_values(by='Mois')     # Trier les données par mois

In [ ]:
data_2023_auz = data_2023.loc[31035001]          # Filtrer les données pour l'AUZ 31035001
data_2023_auz['TM_2023'] = data_2023_auz['TM'] / 10 
data_2023_auz['RR_2023'] = data_2023_auz['RR'] / 10
data_2023_auz.set_index('Mois', inplace=True)     # Définir le mois comme index
data_2023_auz = data_2023_auz[['RR_2023', 'TM_2023']]       # Sélectionner les colonnes d'intérêt
data_2023_auz.to_json(output_dir + 'data_2023_auz.json', orient='table')  # Sauvegarder les données au format JSON
data_2023_auz

In [ ]:
barplot = data_2023_auz.hvplot.bar(
    x='Mois', y='RR', stacked=True, width=600, height=400, label='Pluviométrie (mm)', ylim=(0, data_2023_auz['RR'].max() + 5)
    )      # Créer un graphique en barres pour la pluviométrie
lineplot = data_2023_auz.hvplot.line(
    x='Mois', y='TM', color='red', line_width=2, label='Température moyenne (°C)', ylim=(0, (data_2023_auz['RR'].max() + 5)/2), yaxis='right'
    )      # Créer un graphique en ligne pour la température moyenne
plot = (barplot * lineplot).opts(
    multi_y=True, 
    title="Pluviométrie et Température moyenne (2023)", 
    xlabel='Mois', 
    ylabel='Valeurs', 
    legend_position='top_left'
    )      # Combiner les deux graphiques
plot

# Plot données 1950-2022

In [ ]:
data = pd.read_csv(r"data\MENSQ_31_previous-1950-2022.csv", sep=';', decimal=',', thousands='.', index_col=0)  # Lire le fichier CSV
data.head()

In [ ]:
data_auz = data.loc[31035001]
data_auz.index = pd.to_datetime(data_auz['AAAAMM'].astype(str), format='%Y%m')
data_1950_auz = data_auz[['RR', 'TM']]          # Filtrer les données pour l'AUZ 31035001
data_1950_auz['TM_mean'] = data_1950_auz['TM'] / 10 
data_1950_auz['RR_mean'] = data_1950_auz['RR'] / 10
data_1950_auz

In [ ]:
dt_after_2003 = data_1950_auz.loc['2003':]    # Filtrer les données à partir de 2003 
dt_after_2003

In [ ]:
mean_pre_2022 = dt_after_2003[['TM_mean','RR_mean']].groupby(dt_after_2003.index.month).mean()    # Calculer la moyenne mensuelle
mean_pre_2022.index.name = 'Mois'
mean_pre_2022.to_json(output_dir + 'mean_pre_2022.json', orient='table')    # Sauvegarder les données au format JSON
mean_pre_2022

In [ ]:
barplot = mean_pre_2022.hvplot.bar(
    x='Mois', y='RR', stacked=True, width=600, height=400, label='Pluviométrie moyenne entre 2003 et 2022 (mm)', 
    ylim=(0, mean_pre_2022['RR'].max() + 5), color='yellow'
    )      # Créer un graphique en barres pour la pluviométrie
lineplot = mean_pre_2022.hvplot.line(
    x='Mois', y='TM', color='green', line_width=2, label='Température moyenne entre 2003 et 2022 (mm) (°C)', 
    ylim=(0, (mean_pre_2022['RR'].max() + 5)/2), yaxis='right'
    )      # Créer un graphique en ligne pour la température moyenne
plot2 = (barplot * lineplot).opts(
    multi_y=True, 
    title="Pluviométrie et Température moyenne entre entre 2003 et 2022", 
    xlabel='Mois', 
    ylabel='Valeurs', 
    legend_position='top_left'
    )      # Combiner les deux graphiques
plot2

# Mix des deux tableaux

In [3]:
data_2023 = pd.read_json(r'output\data_2023_auz.json', orient='table')    # Lire les données de 2023
data_pre_2022 = pd.read_json(r'output\mean_pre_2022.json', orient='table')    # Lire les données de 2003 à 2022

In [44]:
table = data_2023.join(data_pre_2022)    # Joindre les deux jeux de données
table

,RR_2023,TM_2023,TM_mean,RR_mean
Mois,,,,
1,79.0,6.1,6.245,69.275000
2,18.0,6.9,7.195,41.650000
3,44.5,11.4,10.110,53.350000
4,58.5,13.3,13.065,61.025000
5,85.0,17.2,16.380,64.475000
6,121.0,22.0,20.805,58.350000
7,12.5,23.2,22.690,36.650000
8,28.5,24.2,22.605,38.078947
9,23.5,22.4,19.550,42.815789


## Barplot

In [87]:
hv.help(hv.Bars)

Bars

Online example: https://holoviews.org/reference/elements/bokeh/Bars.html

-------------
Style Options
-------------

	alpha, bar_width, cmap, color, fill_alpha, fill_color, hover_alpha, hover_color, hover_fill_alpha, hover_fill_color, hover_line_alpha, hover_line_cap, hover_line_color, hover_line_dash, hover_line_dash_offset, hover_line_join, hover_line_width, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, muted, muted_alpha, muted_color, muted_fill_alpha, muted_fill_color, muted_line_alpha, muted_line_cap, muted_line_color, muted_line_dash, muted_line_dash_offset, muted_line_join, muted_line_width, nonselection_alpha, nonselection_color, nonselection_fill_alpha, nonselection_fill_color, nonselection_line_alpha, nonselection_line_cap, nonselection_line_color, nonselection_line_dash, nonselection_line_dash_offset, nonselection_line_join, nonselection_line_width, selection_alpha, selection_color, selection_fill_alpha, selection_fill_color, sel